In [32]:
import os
import cv2

if not os.path.exists('fingers/0'):
    os.makedirs('fingers/0')
if not os.path.exists('fingers/1'):
    os.makedirs('fingers/1')
if not os.path.exists('fingers/2'):
    os.makedirs('fingers/2')
if not os.path.exists('fingers/3'):
    os.makedirs('fingers/3')
if not os.path.exists('fingers/4'):
    os.makedirs('fingers/4')
if not os.path.exists('fingers/5'):
    os.makedirs('fingers/5')

num_0 = 0
num_1 = 0
num_2 = 0
num_3 = 0
num_4 = 0
num_5 = 0

for filename in os.listdir('fingers/train'):
    if filename.endswith(".png"):
        img =  cv2.imread('fingers/train/'+ filename)
        ret,img = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
        img = cv2.resize(img,(100,100))
        name = filename.split('_')[1]
#         print(name)
        if name[0] == '0':
            cv2.imwrite('fingers/0/'+str(num_0)+'_'+name,img)
            num_0 = num_0 + 1
        elif name[0] == '1':
            cv2.imwrite('fingers/1/'+str(num_1)+'_'+name,img)
            num_1 = num_1 + 1
        elif name[0] == '2':
            cv2.imwrite('fingers/2/'+str(num_2)+'_'+name,img)
            num_2 = num_2 + 1
        elif name[0] == '3':
            cv2.imwrite('fingers/3/'+str(num_3)+'_'+name,img)
            num_3 = num_3 + 1
        elif name[0] == '4':
            cv2.imwrite('fingers/4/'+str(num_4)+'_'+name,img)
            num_4 = num_4 + 1
        elif name[0] == '5':
            cv2.imwrite('fingers/5/'+str(num_5)+'_'+name,img)
            num_5 = num_5 + 1
            

In [10]:
if not os.path.exists('fingers/validate/0'):
    os.makedirs('fingers/validate/0')
if not os.path.exists('fingers/validate/1'):
    os.makedirs('fingers/validate/1')
if not os.path.exists('fingers/validate/2'):
    os.makedirs('fingers/validate/2')
if not os.path.exists('fingers/validate/3'):
    os.makedirs('fingers/validate/3')
if not os.path.exists('fingers/validate/4'):
    os.makedirs('fingers/validate/4')
if not os.path.exists('fingers/validate/5'):
    os.makedirs('fingers/validate/5')

In [21]:
import cv2
img = cv2.imread('fingers/train/000cac8e-fcf0-4f8c-bd16-c257d1e6d7a8_2L.png',0)
while(True):
    ret,thresh1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
#     ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cv2.imshow('img',img)
    cv2.imshow('thresh1',thresh1)    
#     cv2.imshow('thresh2',th2)
    img2 = cv2.resize(thresh1,(100,100))
    cv2.imshow('img2',img2)
    
    if (cv2.waitKey(1) & 0xFF == ord('q')):
        break    
cv2.destroyAllWindows()

# Model

In [2]:
import cv2
import numpy as np
import os
y_train = np.empty([0,6])
x_train = np.empty([0,100,100,3])
for folder in os.listdir('fingers/train_pre'):
    print(folder)
    for filename in os.listdir('fingers/train_pre/'+folder):
#         print(filename)
        if filename.endswith(".png"):
            img = cv2.imread('fingers/train_pre/'+folder+'/'+filename,0)
#             print(img.shape)
            np.concatenate((y_train,folder), axis=None)
            np.concatenate((x_train,img), axis=None)

0
1
2
3
4
5


In [3]:
print(y_train.shape)
print(x_train.shape)

(0, 6)
(0, 100, 100, 3)


In [1]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.5,rotation_range = 90,
        zoom_range=0.2,
        horizontal_flip=True,width_shift_range = 0.3,height_shift_range = 0.3)

train_generator = datagen.flow_from_directory(
        'drawcontour/train',
        target_size=(100, 100),shuffle=True,
        batch_size=32,color_mode='rgb',
        class_mode='categorical')

test_generator = datagen.flow_from_directory(
        'drawcontour/test',
        target_size=(100, 100),shuffle=True,
        batch_size=32,color_mode='rgb',
        class_mode='categorical')

validation_generator = datagen.flow_from_directory(
        'drawcontour/val',
        target_size=(100, 100),shuffle=True,
        batch_size=32,color_mode='rgb',
        class_mode='categorical')

Using TensorFlow backend.


Found 7200 images belonging to 3 classes.
Found 1800 images belonging to 3 classes.
Found 1800 images belonging to 3 classes.


In [2]:
print(train_generator.class_indices)
print(test_generator.class_indices)
print(validation_generator.class_indices)

{'hammer': 0, 'paper': 1, 'scissor': 2}
{'hammer': 0, 'paper': 1, 'scissor': 2}
{'hammer': 0, 'paper': 1, 'scissor': 2}


In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense ,BatchNormalization

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100, 100, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

# model.add(Dropout(0.5))

# model.add(Conv2D(128, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(BatchNormalization())
model.add(Flatten())

model.add(Dense(3,activation='softmax'))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
__________

In [5]:
from keras.optimizers import SGD
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

In [6]:
from keras.optimizers import Adam
adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam,metrics=['accuracy'])

In [7]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=300)
model.save('./model/_2/fucking_model3.h5') 

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
2000/2000 [==============================] - 213s 107ms/step - loss: 0.3214 - acc: 0.8905 - val_loss: 0.1471 - val_acc: 0.9507
Epoch 2/10
2000/2000 [==============================] - 205s 103ms/step - loss: 0.1963 - acc: 0.9333 - val_loss: 0.3562 - val_acc: 0.8786
Epoch 3/10
2000/2000 [==============================] - 194s 97ms/step - loss: 0.2041 - acc: 0.9413 - val_loss: 0.0915 - val_acc: 0.9701
Epoch 4/10
2000/2000 [==============================] - 193s 96ms/step - loss: 0.2006 - acc: 0.9456 - val_loss: 0.1674 - val_acc: 0.9608
Epoch 5/10
2000/2000 [==============================] - 197s 98ms/step - loss: 0.1910 - acc: 0.9508 - val_loss: 0.4179 - val_acc: 0.8886
Epoch 6/10
2000/2000 [==============================] - 195s 97ms/step - loss: 0.2142 - acc: 0.9511 - val_loss: 0.4960 - val_acc: 0.8698
Epoch 7/10
2000/2000 [==============================] - 194s 97ms/step - loss: 0.2221 - acc: 0.9513 - val_loss: 0.1363 - val_acc

In [10]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict_generator(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['hammer', 'paper', 'scissor']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

ModuleNotFoundError: No module named 'sklearn'

In [9]:
#model3 fucking_model3.h5 (100,100) drawcontour rotate
scores3 = model.evaluate_generator(test_generator,60)
print(scores3)

[0.38284345467886344, 0.9256329113924051]


In [ ]:
# ---------------------------------------------------------------------------------------------------

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(128, (3, 3), input_shape=(100, 100, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(6,activation='softmax'))

C:\Users\WIN10\Anaconda3\envs\conda1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.optimizers import SGD
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 128)       1280      
_________________________________________________________________
activation_1 (Activation)    (None, 98, 98, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 49, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 32)        36896     
_________________________________________________________________
activation_2 (Activation)    (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 32)        0         
__________

In [5]:
model.load_weights('first_try.h5') 

In [6]:
model.save('first_try.h5')